In [1]:
import httplib2
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.credentials import Credentials
from datetime import datetime
import json

In [11]:

CREDENTIALS_FILE = 'creds.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    CREDENTIALS_FILE,
    ['https://www.googleapis.com/auth/spreadsheets',
     'https://www.googleapis.com/auth/drive'])
httpAuth = credentials.authorize(httplib2.Http())

service = build('sheets', 'v4', http = httpAuth)
service_drive = build('drive', 'v3', http = httpAuth)

spreadsheet_id = '1ZJ6VzQOQJlZvVtJ2UNhb8Uj4El2TV8lG7wMfMwMiJaM'

def get_table_modified_time():
  response = service_drive.files().get(fileId=spreadsheet_id,fields="modifiedTime").execute()
  modified_time_str = response['modifiedTime']
  date = datetime.fromisoformat(modified_time_str[:-1])
  return date

def get_content():
  values = service.spreadsheets().values().get(
    spreadsheetId=spreadsheet_id,
    range='A1:E100',
    majorDimension='ROWS'
  ).execute()
  
  keys = values['values'][0]
  keys = [key.lower() for key in keys]
  videos_row = values['values'][1:]
  videos = []
  
  for row in videos_row:
    video = {}
    for index, value in enumerate(keys):
      if len(row) > index:
          video[value] = row[index]
      else:
          video[value] = ''
        
    if 'link' in video:
      duration = get_video_length(video['link'])
      video['duration'] = duration
      
    videos.append(video)
  
  data = []
  topic_index_dict = {}
  for video in videos:
    topic = video['topic']
    if topic in topic_index_dict:
      index = topic_index_dict[topic]
      data[index]['videos'].append(video)
    else:
      topic_index_dict[topic] = len(data)
      data.append({
        'topic': topic,
        'videos': [
          video
        ]
      })
  return data

In [15]:
values = service.spreadsheets().values().get(
    spreadsheetId=spreadsheet_id,
    range='A1:E100',
    majorDimension='ROWS'
).execute()

values['values']

[['sds'], ['sds', '', 'sd'], ['', 'sd']]

In [14]:
get_table_modified_time()

datetime.datetime(2022, 10, 5, 12, 51, 40, 611000)

In [2]:
from pathlib import Path

In [3]:

class PathConfig:
    storageBaseDir = Path("./DataStorage")
    storageUsersDir = storageBaseDir / "Users"

path = PathConfig()

In [4]:

if path.storageUsersDir.is_dir():
    print("Is dir")
else:
    print("Is not dir")

Is dir


In [12]:
kek = path.storageUsersDir / "azazazadsdz"

In [13]:
kek.mkdir(parents=True, exist_ok=True)

In [3]:
import pytz
from datetime import datetime
# import datetime

In [4]:
tz = pytz.timezone('Europe/Berlin')
now = datetime.now(tz)

In [5]:
def getTimestamp():
    tz = pytz.timezone('Europe/Moscow')
    now = datetime.now(tz)
    return now.strftime("%d.%m.%Y, %H:%M:%S")

In [6]:
getTimestamp()

'31.10.2022, 03:47:35'

In [7]:
datetime.now().isocalendar()[1]

44

In [8]:
datetime(2022, 1, 3).isocalendar()[1]

1

In [47]:
import xlsxwriter
import json
from pathlib import Path
from datetime import date, timedelta


In [43]:
class PathConfig:
    storageBaseDir = Path("./DataStorage")
    storageUsersDir = storageBaseDir / "Users"
    storageBotContentDir = storageBaseDir / "BotContent"
    storageBotContentPrivateConfig = storageBotContentDir / "PrivateConfig.json"

path = PathConfig()

In [19]:
def getJsonData(filePath):
    with filePath.open() as json_file:
        data = json.load(json_file)
    return data

In [29]:

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days + 1)):
        yield start_date + timedelta(n)

In [32]:
dateConfig = getJsonData(path.storageBotContentPrivateConfig)["startDate"]
startDate = date(dateConfig["year"], dateConfig["month"], dateConfig["day"])

In [63]:
# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('TotalHistory.xlsx')
worksheet = workbook.add_worksheet("Start")
eventName = "Старт"

row = 0 
col = 0

worksheet.write(row, col, "Дата / Пользователь")
row += 1

dates = [strDate.strftime("%d.%m.%Y") for strDate in daterange(startDate, date.today())]
for single_date in dates:
    worksheet.write(row, col, single_date)
    row += 1

col += 1

for userFolder in path.storageUsersDir.iterdir():
    row = 0
    history = getJsonData(userFolder / "history.json")
    worksheet.write(row, col, userFolder.name)
    
    row = 1
    for single_date in dates:
        dateEventsCount = len([event for event in history if event["timestamp"]["date"] == single_date and event["event"] == eventName])
        worksheet.write(row, col, dateEventsCount)
        row += 1

    col += 1

workbook.close()

In [51]:
path.storageUsersDir.absolute()

PosixPath('/Users/antontekutov/Documents/Projects/BotDevelopment/NurturanceBot/DataStorage/Users')

In [56]:
glob("DataStorage/Users//*")

['DataStorage/Users/5585413083', 'DataStorage/Users/243378934']

In [65]:
import enum

In [66]:
class HistoryEvent(enum.Enum):

    start = "Старт"
    sendMessage = "Отправил сообщение"

In [68]:
HistoryEvent.start.name

'start'